In [81]:
import json
from sklearn.preprocessing import LabelEncoder
import pythainlp
import numpy as np

In [4]:
with open('intents copy.json',encoding='utf-8') as file:
    data = json.load(file)

In [6]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [13]:
encoder = LabelEncoder()
training_labels  = encoder.fit_transform(training_labels)

In [51]:
from pythainlp import word_tokenize

tokens = []
list_tokens = [word_tokenize(w) for w in training_sentences]
for list_token in list_tokens:
    for word in list_token:
        if word not in tokens:
            tokens.append(word)

In [69]:
word_index = {}
index_word = {}
for i,j in enumerate(tokens):
    word_index[j] = i
    index_word[i] = j

train_sequences = []
for sentence in training_sentences:
    train_sequence = []
    temp = word_tokenize(sentence)
    for i in temp:
        if i in word_index:
            train_sequence.append(word_index[i])
        else:
            train_sequence.append('<oov>')
    train_sequences.append(train_sequence)

    

In [15]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [75]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [76]:
max_len = 20
padded_sequences = pad_sequences(train_sequences, truncating='post', maxlen=max_len)

In [78]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim ,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [79]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 16595 (64.82 KB)
Trainable params: 16595 (64.82 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [82]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
1/1 [==============================] - 0s 330ms/step - loss: 1.0982 - accuracy: 0.3889
Epoch 2/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0973 - accuracy: 0.3333
Epoch 3/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0963 - accuracy: 0.3333
Epoch 4/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0954 - accuracy: 0.3889
Epoch 5/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0947 - accuracy: 0.3889
Epoch 6/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0939 - accuracy: 0.3889
Epoch 7/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0932 - accuracy: 0.3889
Epoch 8/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0924 - accuracy: 0.3889
Epoch 9/550
1/1 [==============================] - 0s 4ms/step - loss: 1.0916 - accuracy: 0.3889
Epoch 10/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0908 - accuracy: 0.3889
Epoch 11/550
1/1 [=========